In [36]:
# data from Bench4BL
import pandas as pd

data = pd.DataFrame()
data["Apache_CAMEL"] = "test"

In [7]:
import xgboost as xgb
import pandas as pd

projectVersionMap = ["Apache_CAMEL","Apache_HBASE","Apache_HIVE","Commons_CODEC","Commons_COLLECTIONS","Commons_COMPRESS","Commons_CONFIGURATION","Commons_CRYPTO","Commons_CSV","Commons_IO","Commons_LANG","Commons_MATH","Commons_WEAVER","JBoss_ENTESB","JBoss_JBMETA","Previous_AspectJ","Previous_PDE","Previous_SWT","Previous_ZXing","Spring_AMQP","Spring_ANDROID","Spring_BATCHADM","Spring_BATCH","Spring_DATACMNS","Spring_DATAGRAPH","Spring_DATAJPA","Spring_DATAMONGO","Spring_DATAREDIS","Spring_DATAREST","Spring_LDAP","Spring_MOBILE","Spring_ROO","Spring_SECOAUTH","Spring_SEC","Spring_SGF","Spring_SHDP","Spring_SHL","Spring_SOCIALFB","Spring_SOCIALLI","Spring_SOCIALTW","Spring_SOCIAL","Spring_SPR","Spring_SWF","Spring_SWS","Wildfly_ELY","Wildfly_SWARM","Wildfly_WFARQ","Wildfly_WFCORE","Wildfly_WFLY","Wildfly_WFMP"]
print(len(projectVersionMap))


50


In [14]:
for key in projectVersionMap:
    print("Project " + key)
    gbm = xgb.Booster(model_file='model/' + key + '.model')
    
    test = pd.read_csv('data_old/' + key + '.csv')    
    columns = test.drop(['project','bug','Result'], axis=1).columns
    result = pd.DataFrame(test)
    result['Result2'] = gbm.predict(xgb.DMatrix(test[columns]))
    result.to_csv('results/' + key + '.csv', index=False, sep=',')

Project Apache_CAMEL
Project Apache_HBASE
Project Apache_HIVE
Project Commons_CODEC
Project Commons_COLLECTIONS
Project Commons_COMPRESS
Project Commons_CONFIGURATION
Project Commons_CRYPTO
Project Commons_CSV
Project Commons_IO
Project Commons_LANG
Project Commons_MATH
Project Commons_WEAVER
Project JBoss_ENTESB
Project JBoss_JBMETA
Project Previous_AspectJ
Project Previous_PDE
Project Previous_SWT
Project Previous_ZXing
Project Spring_AMQP
Project Spring_ANDROID
Project Spring_BATCHADM
Project Spring_BATCH
Project Spring_DATACMNS
Project Spring_DATAGRAPH
Project Spring_DATAJPA
Project Spring_DATAMONGO
Project Spring_DATAREDIS
Project Spring_DATAREST
Project Spring_LDAP
Project Spring_MOBILE
Project Spring_ROO
Project Spring_SECOAUTH
Project Spring_SEC
Project Spring_SGF
Project Spring_SHDP
Project Spring_SHL
Project Spring_SOCIALFB
Project Spring_SOCIALLI
Project Spring_SOCIALTW
Project Spring_SOCIAL
Project Spring_SPR
Project Spring_SWF
Project Spring_SWS
Project Wildfly_ELY
Project

In [8]:
def evaluate_for_key(key, bugs_length=None):
    resultData = pd.read_csv('results/' + key + '.csv')     
    bugs = resultData.bug.unique()
    print("Project " + key)
    if bugs_length == None:
        bugs_length = len(bugs)
    print(bugs_length)
    count1 = 0;
    count5 = 0;
    count10 = 0;
    mapValue = 0;
    mrr = 0;
    for bug in bugs:
        subData = resultData[resultData['bug'] == bug]
        subDataSorted = subData.sort_values('Result2',ascending=False)
        
        if subDataSorted.iloc[0,-2] == 1:
            count1 += 1;
        if 1 in subDataSorted.values[0:5,-2]:
            count5 += 1;
        if 1 in subDataSorted.values[0:10,-2]:
            count10 += 1;
        
        summe = 0;
        rr = 0;
        retrieved_d = 0;
        i = 0
        c = 0
        for resultV in subDataSorted.values[:,-2]:
            if resultV == 1 and c < 10:
                c += 1
                retrieved_d += 1;
                summe += (retrieved_d /(i+1))
            if resultV == 1 and rr == 0:    
                rr += (1/ (i+1))
            i += 1
        if c != 0:
            mapValue += (summe/c)
        else: 
            print("c zero!" + str(bug))
            mapValue +=0
            #rr = 1
        mrr += rr
    return count1/bugs_length,count5/bugs_length,count10/bugs_length,mapValue/bugs_length,mrr/bugs_length

In [9]:
import statistics

maps = []
mrrs = []
top1s = []
top5s = []
top10s = []

for key in projectVersionMap:
    count1, count5, count10, mapValue, mrr = evaluate_for_key(key)
    print("Count 1: " + str(count1) + " Count 5: " + str(count5) + " Count 10:" + str(count10))
    print("MAP " + str(mapValue))
    print("MRR " + str(mrr))
    
    top1s.append(count1)
    top5s.append(count5)
    top10s.append(count10)
    maps.append(mapValue)
    mrrs.append(mrr)


Project Apache_CAMEL
1147
c zero!1803
c zero!2209
c zero!1442
c zero!1630
c zero!1671
c zero!1755
c zero!1839
c zero!1870
c zero!1917
c zero!1927
c zero!1957
c zero!2251
c zero!2318
c zero!2447
c zero!3082
c zero!8315
c zero!7320
c zero!7772
c zero!8214
c zero!9202
c zero!8189
c zero!9205
c zero!8690
c zero!9101
c zero!9281
c zero!2485
c zero!2639
c zero!2671
c zero!2740
c zero!3193
c zero!3752
c zero!3104
Count 1: 0.34524847428073235 Count 5: 0.5649520488230165 Count 10:0.6573670444638187
MAP 0.3659457139842377
MRR 0.4495727355357287
Project Apache_HBASE
858
c zero!437
c zero!444
c zero!428
c zero!281
c zero!516
c zero!524
c zero!514
c zero!497
c zero!550
c zero!552
c zero!564
c zero!577
c zero!680
c zero!686
c zero!694
c zero!707
c zero!716
c zero!744
c zero!756
c zero!764
c zero!812
c zero!858
c zero!500
c zero!1039
c zero!1238
c zero!1107
c zero!1352
c zero!1600
c zero!14431
c zero!14343
c zero!16061
c zero!13722
c zero!14011
c zero!14660
c zero!15063
c zero!15231
c zero!15578
c ze

In [17]:



print("Average MAP: " + str(statistics.mean(maps)))
print("Average MRR: " + str(statistics.mean(mrrs)))
print("Average 1: " + str(statistics.mean(top1s)))
print("Average 5: " + str(statistics.mean(top5s)))
print("Average 10: " + str(statistics.mean(top10s)))

Average MAP: 0.4794944485824161
Average MRR: 0.5413109595619753
Average 1: 0.43816702794009105
Average 5: 0.6766159172236262
Average 10: 0.7437554599617531


In [12]:
top10s

[0.6573670444638187,
 0.7412587412587412,
 0.6365873666940115,
 0.975609756097561,
 0.8494623655913979,
 0.9714285714285714,
 0.8962962962962963,
 0.5,
 0.9285714285714286,
 0.9431818181818182,
 0.8663366336633663,
 0.8035714285714286,
 1.0,
 0.20512820512820512,
 0.4230769230769231,
 0.32867132867132864,
 0.55,
 0.7244897959183674,
 0.6,
 0.8712871287128713,
 0.6666666666666666,
 0.8421052631578947,
 0.7742718446601942,
 0.7483870967741936,
 0.3263157894736842,
 0.8783783783783784,
 0.8390804597701149,
 0.9166666666666666,
 0.8257575757575758,
 0.8113207547169812,
 0.9090909090909091,
 0.7312588401697313,
 0.6666666666666666,
 0.7781885397412199,
 0.7714285714285715,
 0.8536585365853658,
 0.8888888888888888,
 0.8666666666666667,
 1.0,
 0.875,
 0.7333333333333333,
 0.6111111111111112,
 0.7164179104477612,
 0.6744186046511628,
 0.30434782608695654,
 0.6071428571428571,
 1.0,
 0.6076696165191741,
 0.4912087912087912,
 1.0]

In [44]:
import pandas as pd

data = pd.read_csv('model_data_compress.csv')
projects = data.project.unique()
projectVersionMap = {}
for project in projects:
    projectParts = project.split("_")
    groupName = projectParts[0] + "_" + projectParts[1]
    if groupName not in projectVersionMap:
        projectVersionMap[groupName] = []
    projectVersionMap[groupName].append(project)

projectVersionMap

{'smartshark_ant-ivy': ['smartshark_ant-ivy_1.4.1',
  'smartshark_ant-ivy_2.0.0',
  'smartshark_ant-ivy_2.1.0',
  'smartshark_ant-ivy_2.2.0',
  'smartshark_ant-ivy_2.3.0',
  'smartshark_ant-ivy_2.4.0'],
 'smartshark_archiva': ['smartshark_archiva_1.0',
  'smartshark_archiva_1.1',
  'smartshark_archiva_1.2',
  'smartshark_archiva_1.3',
  'smartshark_archiva_2.0.0',
  'smartshark_archiva_2.1.0',
  'smartshark_archiva_2.2.0'],
 'smartshark_calcite': ['smartshark_calcite_1.0.0',
  'smartshark_calcite_1.1.0',
  'smartshark_calcite_1.10.0',
  'smartshark_calcite_1.11.0',
  'smartshark_calcite_1.12.0',
  'smartshark_calcite_1.13.0',
  'smartshark_calcite_1.14.0',
  'smartshark_calcite_1.15.0',
  'smartshark_calcite_1.2.0',
  'smartshark_calcite_1.3.0',
  'smartshark_calcite_1.4.0',
  'smartshark_calcite_1.5.0',
  'smartshark_calcite_1.6.0',
  'smartshark_calcite_1.7.0',
  'smartshark_calcite_1.8.0',
  'smartshark_calcite_1.9.0'],
 'smartshark_cayenne': ['smartshark_cayenne_3.0.0',
  'smartsha

In [45]:
import os
import pandas as pd

data_mp = pd.read_csv('smartshark_other/statistic.csv')


data = pd.DataFrame()
for file in os.listdir("smartshark_other/amalgam"):
    project, version = file.replace('smartshark_','').replace('_output.txt','').split("_")
    element = data_mp.loc[(data_mp['Project'] == "smartshark_" + project) & (data_mp['Version'] == version), 'Bugs'].values[0]
    data = data.append({"project" : project, "version" : version, "bugs": element}, ignore_index=True)

agg_funcs = {'bugs' : 'sum', 
         'version' : 'count'
}

data = data.groupby("project").agg(agg_funcs)

In [50]:
maps = []
mrrs = []
top1s = []
top5s = []
top10s = []

    
gbm = xgb.Booster(model_file='model/smartshark2.model')
for key in projectVersionMap:
    print(key)
    test = pd.read_csv('data/' + key +'.csv')    
    columns = test.drop(['project','bug','Result'], axis=1).columns
    result = pd.DataFrame(test)
    result['Result2'] = gbm.predict(xgb.DMatrix(test[columns]))
    result.to_csv('results/' + key +'.csv', index=False, sep=',')
    
    pro = key.replace('smartshark_','')
    
    count1, count5, count10, mapValue, mrr = evaluate_for_key(key)
    print("Count 1: " + str(count1) + " Count 5: " + str(count5) + " Count 10:" + str(count10))
    print("MAP " + str(mapValue))
    print("MRR " + str(mrr))
        
    top1s.append(count1)
    top5s.append(count5)
    top10s.append(count10)
    maps.append(mapValue)
    mrrs.append(mrr)
    

smartshark_ant-ivy
Project smartshark_ant-ivy
431
Count 1: 0.5986078886310905 Count 5: 0.8445475638051044 Count 10:0.9025522041763341
MAP 0.559783526259683
MRR 0.7013264823355522
smartshark_archiva
Project smartshark_archiva
275
Count 1: 0.509090909090909 Count 5: 0.7709090909090909 Count 10:0.8109090909090909
MAP 0.45234951380707394
MRR 0.621760516841609
smartshark_calcite
Project smartshark_calcite
603
Count 1: 0.6683250414593698 Count 5: 0.87893864013267 Count 10:0.9270315091210614
MAP 0.5141707004688167
MRR 0.7631367763464648
smartshark_cayenne
Project smartshark_cayenne
328
Count 1: 0.4847560975609756 Count 5: 0.6646341463414634 Count 10:0.7439024390243902
MAP 0.4194188952076058
MRR 0.5731052429005916
smartshark_commons-bcel
Project smartshark_commons-bcel
29
Count 1: 0.6551724137931034 Count 5: 0.896551724137931 Count 10:0.9655172413793104
MAP 0.6399087933223578
MRR 0.7539956212370005
smartshark_commons-beanutils
Project smartshark_commons-beanutils
39
Count 1: 0.717948717948718 

In [35]:
import statistics 

print("Average MAP: " + str(statistics.median(maps)))
print("Average MRR: " + str(statistics.median(mrrs)))
print("Average 1: " + str(statistics.mean(top1s)))
print("Average 5: " + str(statistics.mean(top5s)))
print("Average 10: " + str(statistics.mean(top10s)))

Average MAP: 0.5307317163564318
Average MRR: 0.7541899201843119
Average 1: 0.6681318606012456
Average 5: 0.8169281531015018
Average 10: 0.8397582941600291


In [44]:
sum(maps)/sum(data['bugs'])
Average MAP: 0.5821443800289604
Average MRR: 0.7551110616559689
Average 1: 0.6435088540648778
Average 5: 0.8512669335925249
Average 10: 0.89962887419386

0.0817266315636614

In [54]:
top5s

[0.8445475638051044,
 0.7709090909090909,
 0.87893864013267,
 0.6646341463414634,
 0.896551724137931,
 0.9230769230769231,
 0.9714285714285714,
 0.96,
 0.916083916083916,
 0.9354838709677419,
 0.9279279279279279,
 0.8571428571428571,
 1.0,
 0.8461538461538461,
 0.8596491228070176,
 0.9740740740740741,
 0.9069069069069069,
 0.96875,
 1.0,
 1.0,
 0.9236641221374046,
 0.755,
 0.743421052631579,
 0.8114035087719298,
 0.8837209302325582,
 0.5433526011560693,
 0.7364620938628159,
 0.7356020942408377,
 0.7601476014760148,
 0.8210526315789474,
 0.8368794326241135,
 0.8498233215547704,
 0.8206896551724138,
 0.8235294117647058,
 0.8493150684931506,
 0.7023498694516971,
 0.8670634920634921,
 0.7824074074074074]